In [ ]:
from gql import Client, gql
from gql.transport.requests import RequestsHTTPTransport

class OrderContextServer(MCPBaseServer):
    def __init__(self):
        super().__init__("order-context", "order_history")
        self.transport = RequestsHTTPTransport(
            url="https://graphql.ordersystem.com",
            headers=self._get_auth_headers()
        )

    def _get_auth_headers(self):
        # GenAI-generated from GraphQL schema
        token = self.secret_client.access_secret_version(
            name="projects/my-project/secrets/GRAPHQL_TOKEN/versions/latest"
        ).payload.data.decode()
        return {"Authorization": f"Bearer {token}"}

    def authenticate(self, request):
        # OAuth 2.1 with order_history scope
        token = request.token
        claims = jwt.decode(token, options={"verify_signature": False})
        if "order_history" not in claims["scope"]:
            raise PermissionError("Missing required scope")

    def fetch_data(self, request):
        # Parameterized GraphQL query (GenAI-generated)
        client = Client(transport=self.transport, fetch_schema_from_transport=True)
        query = gql("""
            query GetOrders($userId: ID!, $limit: Int = 10) {
                orders(userId: $userId, first: $limit) {
                    edges {
                        node {
                            id
                            total
                            items
                            created
                        }
                    }
                }
            }
        """)
        return client.execute(query, variable_values={
            "userId": request.json["user_id"],
            "limit": request.json.get("limit", 10)
        })

    def build_context(self, raw_data):
        # Convert to JSON-LD
        orders = [edge["node"] for edge in raw_data["orders"]["edges"]]
        return {
            "@context": "https://schema.org/",
            "@type": "ItemList",
            "itemListElement": [
                {
                    "@type": "Order",
                    "orderId": o["id"],
                    "price": o["total"],
                    "orderDate": o["created"]
                } for o in orders
            ],
            "metadata": {
                "source": "GraphQL",
                "freshness": datetime.datetime.utcnow().isoformat(),
                "query_depth": 2  # Policy-enforced
            }
        }